In [2]:
import tkinter as tk
from tkinter import Label, Entry, Button, messagebox, Menu

import random
import time

def get_words():
    words = ["jabłko", "banan", "pomarańcza", "truskawka", "gruszka", "winogrono", "kiwi", "mango", "malina", "arbuz", "śliwka", "ananas", "brzoskwinia", "grejpfrut", "mandarynka"]
    words = sorted(words)
    return words

def update_timer():
    global start_time, timer_label, timer_running, czas
    if timer_running:
        
        czas = 120
        
        current_time = time.monotonic()
        elapsed_time = int(czas - (current_time - start_time))
        timer_label.config(text="Czas: {} sekundy".format(elapsed_time))
        if elapsed_time <= 0:
            messagebox.showinfo("Przegrana!", "Czas minął. Nie udało ci się zgadnąć słowa.")
            window.destroy()
        else:
            window.after(100, update_timer)

def update_word_label():
    global chosen_word, word_label
    word_label.config(text="Aktualne słowo: " + "_ " * len(chosen_word))

def calculate_points(elapsed_time):
    return max(0, czas - elapsed_time)

def check_guess(words):
    global chosen_word, attempts, entry, window, timer_label, result_label, attempts_label, max_attempts
    
    attempts += 1
    
    user_guess = entry.get().lower()

    if user_guess not in words:
        messagebox.showwarning("Ostrzeżenie", "Nieprawidłowe słowo. Wybierz słowo z listy.")
        result_label.config(text="Nieprawidłowe słowo. Wybierz słowo z listy.", fg="red")
        
        if timer_running:
            attempts_label.config(text="Liczba prób: {}/{}".format(attempts, max_attempts))
        else:
            attempts_label.config(text="Liczba prób: {}".format(attempts))
        
        if timer_running and attempts >= max_attempts:
            messagebox.showinfo("Przegrana!", "Osiągnięto maksymalną liczbę prób. Nie udało ci się zgadnąć słowa.")
            window.destroy()
        return

    if user_guess < chosen_word:
        result_label.config(text="Wpisane słowo znajduje się przed szukanym słowem", fg="red")
    elif user_guess > chosen_word:
        result_label.config(text="Wpisane słowo znajduje się po szukanym słowie", fg="red")
    else:
        end_time = time.monotonic()
        elapsed_time = int(end_time - start_time)
        points = calculate_points(elapsed_time)
        if 2 <= elapsed_time <= 4 and timer_running:
            messagebox.showinfo("Gratulacje!", "Słowo odgadnięte poprawnie w {} sekundy! \nOtrzymujesz {} punktów.".format(elapsed_time, points))
        elif elapsed_time == 1 and timer_running:
            messagebox.showinfo("Gratulacje!", "Słowo odgadnięte poprawnie w {} sekundę! \nOtrzymujesz {} punktów.".format(elapsed_time, points))
        elif elapsed_time > 4 and timer_running:
            messagebox.showinfo("Gratulacje!", "Słowo odgadnięte poprawnie w {} sekund! \nOtrzymujesz {} punktów.".format(elapsed_time, points))
        else:
            messagebox.showinfo("Gratulacje!", "Słowo odgadnięte poprawnie w {} sekund!".format(elapsed_time))
        
        window.destroy()

    if timer_running:
        attempts_label.config(text="Liczba prób: {}/{}".format(attempts, max_attempts))
    else:
        attempts_label.config(text="Liczba prób: {}".format(attempts))

    # Czyścimy pole wprowadzania tylko gdy słowo jest w liście
    entry.delete(0, tk.END)

    if timer_running and attempts >= max_attempts:
        messagebox.showinfo("Przegrana!", "Osiągnięto maksymalną liczbę prób. Nie udało ci się zgadnąć słowa.")
        window.destroy()

def new_game():
    global chosen_word, attempts, entry, start_time, window, timer_label, result_label, attempts_label, timer_running
    chosen_word = random.choice(get_words())
    attempts = 0
    entry.delete(0, tk.END)
    result_label.config(text="")
    timer_running = False
    timer_label.config(text="Czas: Brak limitu")
    update_word_label()  # Dodano aktualizację etykiety z aktualnym słowem
    attempts_label.config(text="Liczba prób: {}".format(attempts))

def timed_game():
    global chosen_word, attempts, entry, start_time, window, timer_label, result_label, attempts_label, timer_running, max_attempts
    chosen_word = random.choice(get_words())
    attempts = 0
    entry.delete(0, tk.END)
    result_label.config(text="")
    start_time = time.monotonic()
    timer_running = True
    timer_label.config(text="Czas: 120 sekundy")
    update_timer()
    update_word_label()  # Dodano aktualizację etykiety z aktualnym słowem
    attempts_label.config(text="Liczba prób: {}/{}".format(0, max_attempts))

def display_words():
    words = get_words()
    words_text = "\n".join(["- " + word for word in words])
    messagebox.showinfo("Dostępne słowa", f"Dostępne słowa do wpisania:\n{words_text}")


def set_max_attempts(value):
    global max_attempts
    max_attempts = value
    timed_game()

def create_interface():
    global chosen_word, attempts, entry, start_time, window, timer_label, result_label, attempts_label, timer_running
    chosen_word = random.choice(get_words())
    attempts = 0
    timer_running = False

    window = tk.Tk()
    window.title("Gra w zgadywanie słowa")
    window.geometry("400x300")

    # Menu
    menubar = Menu(window)
    window.config(menu=menubar)

    file_menu = Menu(menubar, tearoff=0)
    menubar.add_cascade(label="Gra", menu=file_menu)
    file_menu.add_command(label="Nowa gra", command=new_game)

    # Zagnieżdżone menu dla nowej gry z limitem prób i czasu
    timed_menu = Menu(file_menu, tearoff=0)
    file_menu.add_cascade(label="Nowa gra z limitem prób i czasu", menu=timed_menu)

    # Opcja wyświetlania dostępnych słów
    file_menu.add_command(label="Wyświetl dostępne słowa     F1", command=display_words)

    # Opcje liczby prób
    timed_menu.add_command(label="5 prób", command=lambda: set_max_attempts(5))
    timed_menu.add_command(label="10 prób", command=lambda: set_max_attempts(10))
    timed_menu.add_command(label="15 prób", command=lambda: set_max_attempts(15))

    file_menu.add_separator()
    file_menu.add_command(label="Wyjście", command=window.destroy)

    title_label = Label(window, text="Gra w zgadywanie słowa", font=("Arial", 20))
    title_label.pack(pady=10)

    global word_label
    word_label = Label(window, text="Aktualne słowo: " + "_ " * len(chosen_word), font=("Arial", 14))
    word_label.pack()

    global attempts_label
    attempts_label = Label(window, text="Liczba prób: {}".format(attempts), font=("Arial", 12))
    attempts_label.pack()

    entry = Entry(window, font=("Arial", 12))
    entry.pack(pady=10)

    submit_button = Button(window, text="Zatwierdź", command=lambda: check_guess(get_words()), font=("Arial", 12))
    submit_button.pack()

    window.bind('<Return>', lambda event: check_guess(get_words()))

    window.bind('<F1>', lambda event: display_words())

    window.bind('<Escape>', lambda event: on_closing())

    global result_label
    result_label = Label(window, text="", font=("Arial", 12))
    result_label.pack()

    global timer_label
    timer_label = Label(window, text="Czas: Brak limitu", font=("Arial", 12))
    timer_label.pack()

    start_time = time.monotonic()
    update_timer()

    def on_closing():
        if messagebox.askokcancel("Wyjście", "Czy na pewno chcesz zakończyć grę?"):
            window.destroy()

    window.protocol("WM_DELETE_WINDOW", on_closing)
    window.mainloop()

create_interface()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Piotrek\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Piotrek\AppData\Local\Temp\ipykernel_772\137053969.py", line 171, in <lambda>
    window.bind('<Return>', lambda event: check_guess(get_words()))
                                          ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Piotrek\AppData\Local\Temp\ipykernel_772\137053969.py", line 77, in check_guess
    attempts_label.config(text="Liczba prób: {}".format(attempts))
  File "C:\Users\Piotrek\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1702, in configure
    return self._configure('configure', cnf, kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Piotrek\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1692, in _configure
    self.tk.call(_flatten((self._w, cmd)) 